# **Zamontowanie dysku google**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Import biblioteki TensorFlow**

In [2]:
import tensorflow as tf

# **Wczytanie zbioru uczącego oraz testowego**

In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=360,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.1,
    shear_range=0.2
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/casting_data/casting_data/train',
    target_size=(300, 300),
    batch_size=32,
    color_mode='grayscale',
    class_mode='binary')

test_data = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/casting_data/casting_data/test',
    target_size=(300, 300),
    batch_size=32,
    color_mode='grayscale',
    class_mode='binary')

Found 6637 images belonging to 2 classes.
Found 715 images belonging to 2 classes.


# **Stworzenie modelu sieci neuronowej**

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, 
                           kernel_size=(3,3), 
                           input_shape=(300, 300, 1), 
                           activation='relu'),
                           
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    tf.keras.layers.Conv2D(filters=32, 
                           kernel_size=(3,3), 
                           activation='relu'),

    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(units=64, 
                          activation='relu'),

    tf.keras.layers.Dropout(rate=0.3),
    
    tf.keras.layers.Dense(units=32, 
                          activation='relu'),

    tf.keras.layers.Dropout(rate=0.3),
    
    tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 170528)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                10913856  
_________________________________________________________________
dropout (Dropout)            (None, 64)                0

# **Kompilacja modelu**

In [5]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

# **Automatyczne zapisywanie modelu z najwyższą precyzją**

In [6]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('./', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# **Trenowanie sieci neuronowej**

In [ ]:
model.fit_generator(
    train_data,
    epochs=30,
    validation_data=test_data,
    callbacks=[checkpoint])